## ***Sujet*** : Recherche de recettes et comparaison des prix des produits dans les supermarchés français : Auchan et Carrefour

## Objectif du projet
Ce projet se concentre sur la construction d'une application permettant de rechercher des recettes et de comparer les prix des ingrédients nécessaires dans différents supermarchés (par exemple, Auchan, Carrefour). Avec une large gamme de produits et des prix fluctuants, cet outil offre aux utilisateurs un moyen pratique de planifier leurs repas et d'optimiser leurs dépenses d'épicerie.

L'application est construite sur une interface conviviale et utilise des techniques de traitement du langage naturel (NLP) pour faire correspondre les ingrédients des recettes avec les produits disponibles dans les catalogues des magasins.

Pour ce faire, nous avons collecté les données nécessaires en effectuant un **scraping** des sites web des deux supermarchés et de Marmiton.


## Scraping Auchan

Pour le site d'Auchan, nous avons effectué un scraping des produits disponibles afin de récupérer des informations détaillées sur les produits.

### Étapes principales :
1. Initialisation du **WebDriver** avec Selenium pour naviguer sur le site.
2. Sélection de la localisation (**Strasbourg**) pour adapter les produits disponibles.
3. Parcours des différentes pages de promotions.
4. Extraction des informations suivantes pour chaque produit :
   - **Nom**
   - **Prix**
   - **Unité** (ex : kg, L, etc.)
   - **Pourcentage de réduction**
5. Sauvegarde des données dans un fichier **CSV** pour une analyse ultérieure.

Le processus assure une récupération complète et organisée des promotions pour une localisation spécifique.


In [7]:
import pandas as pd
import selenium
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Initialiser le navigateur Web
driver = webdriver.Chrome()

# Accéder au site web d'Auchan
driver.get('https://www.auchan.fr/boutique/promos')  

# Attendre que le bouton "Rejeter les cookies" soit visible et cliquer dessus
try:
    reject_cookies_button = WebDriverWait(driver, 40).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Continuer sans accepter')]"))
    )
    reject_cookies_button.click()
    print("Cookies rejetés.")
except Exception as e:
    print("Pas de bouton de rejet des cookies trouvé : ", e)
    

# Wait until the element is clickable
afficher_prix_button = WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "product-thumbnail__see-prices-button"))
)

driver.execute_script("arguments[0].scrollIntoView(true);", afficher_prix_button)
time.sleep(1)  # Optionally wait a bit after scrolling

driver.execute_script("arguments[0].click();", afficher_prix_button)


# Attendre que la barre de recherche soit visible
search_bar = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "input.shadow--light.journey__search-input.journeySearchInput"))
)

# Taper "67000" dans la barre de recherche
search_bar.send_keys("67000")

# Attendre que les suggestions apparaissent et sélectionner "Strasbourg"
suggestions_list = WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul[role='listbox'] li"))
)

for suggestion in suggestions_list:
    if "Strasbourg" in suggestion.text:
        suggestion.click()
        break

# Pause pour attendre le chargement de la page suivante
time.sleep(5)


# Recherche de l'élément contenant "Auchan Click&collect G.G. Les Halles - Strasbourg"
try:
    store_to_select = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Auchan Click&collect G.G. Les Halles - Strasbourg')]"))
    )
    # Scroller jusqu'à cet élément
    driver.execute_script("arguments[0].scrollIntoView(true);", store_to_select)

    # Chercher directement le bouton "Choisir" associé, en partant de l'élément textuel et descendant dans la hiérarchie
    choose_button = store_to_select.find_element(By.XPATH, ".//following::button[contains(@class, 'btnJourneySubmit')]")
    
    # Cliquer sur le bouton "Choisir"
    driver.execute_script("arguments[0].click();", choose_button)
    
    print("Bouton 'Choisir' cliqué.")
except Exception as e:
    print(f"Erreur lors de la sélection du magasin : {e}")


Cookies rejetés.
Bouton 'Choisir' cliqué.


In [ ]:
# Listes pour stocker les informations
product_names = []
product_prices = []
product_units = []
discount_percentages = []

# URL de base pour la boutique Auchan avec les promotions
base_url = "https://www.auchan.fr/boutique/promos?page="
page_num = 1  # Commencer à la première page

# Boucle pour scraper toutes les pages
while True:
    print(f"Scraping page {page_num}")
    driver.get(base_url + str(page_num))  # Charger la page avec le numéro actuel

    # Attendre que la page se charge complètement
    time.sleep(3)
    
    # Récupérer les produits sur la page actuelle
    products = driver.find_elements(By.XPATH, "//article[contains(@class, 'product-thumbnail')]")
    
    if not products:
        print(f"Aucun produit trouvé sur la page {page_num}. Fin de la pagination.")
        break
    
    print(f"Nombre de produits trouvés sur la page {page_num}: {len(products)}")
    
    # Récupérer les informations des produits sur la page actuelle
    for p in products:  
        # Récupérer le nom du produit
        try:
            name_element = p.find_element(By.CLASS_NAME, "product-thumbnail__description")
            product_names.append(name_element.text.strip())
        except Exception as e:
            print(f"Erreur lors de la récupération du nom du produit : {e}")
            product_names.append(None)

        # Récupérer le prix du produit
        try:
            price_element = p.find_element(By.CLASS_NAME, "product-thumbnail__price")
            product_prices.append(price_element.text.strip())
        except Exception as e:
            print(f"Erreur lors de la récupération du prix : {e}")
            product_prices.append(None)

        # Récupérer l'unité du produit (g, kg, L, etc.)
        try:
            unit_element = p.find_element(By.CLASS_NAME, "product-attribute")
            product_units.append(unit_element.text.strip())
        except Exception as e:
            print(f"Erreur lors de la récupération de l'unité : {e}")
            product_units.append(None)

        # Récupérer le pourcentage de réduction
        try:
            discount_element = p.find_element(By.CLASS_NAME, "product-discount-label")
            discount_percentages.append(discount_element.text.strip())
        except Exception as e:
            print(f"Erreur lors de la récupération du pourcentage de réduction : {e}")
            discount_percentages.append(None)

    # Passer à la page suivante
    page_num += 1

    # Optionnel : Limiter à un certain nombre de pages (ici, par exemple, 10 pages)
    # if page_num > 10:
    #     break

# Fermer le driver après le scraping
#driver.quit()

# Sauvegarder les informations extraites dans un fichier CSV
with open('Auchan_products.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Nom du Produit', 'Prix', 'Unité', 'Pourcentage de Réduction'])
    for i in range(len(product_names)):
        writer.writerow([product_names[i], product_prices[i], product_units[i], discount_percentages[i]])

print("Scraping terminé et fichier CSV créé avec succès.")


Scraping page 1
Nombre de produits trouvés sur la page 1: 14
Erreur lors de la récupération du pourcentage de réduction : Message: no such element: Unable to locate element: {"method":"css selector","selector":".product-discount-label"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6B9A538A5+3004357]
	(No symbol) [0x00007FF6B96E9970]
	(No symbol) [0x00007FF6B959582A]
	(No symbol) [0x00007FF6B95E5B8E]
	(No symbol) [0x00007FF6B95E5E7C]
	(No symbol) [0x00007FF6B95D93DC]
	(No symbol) [0x00007FF6B960BC1F]
	(No symbol) [0x00007FF6B95D92A6]
	(No symbol) [0x00007FF6B960BDF0]
	(No symbol) [0x00007FF6B962BA4C]
	(No symbol) [0x00007FF6B960B983]
	(No symbol) [0x00007FF6B95D7628]
	(No symbol) [0x00007FF6B95D8791]
	GetHandleVerifier [0x00007FF6B9A7A00D+3161901]
	GetHandleVerifier [0x00007FF6B9ACE060+3506048]
	GetHandleVer

### Nous avons enregistré notre base de données dans un dataframe et visualisé l'architecture des données

In [9]:
df = pd.DataFrame({
    'Nom': product_names,
    'Prix': product_prices,
    'Unité': product_units,
    'Réduction': discount_percentages,
    
})

df

,Nom,Prix,Unité,Réduction
0,LAY'S Biscuits soufflés 3D's goût nature,"2,75€",300g,-50% sur le 2ème
1,LAY'S Chips ondulées recette paysanne nature,"3,49€",370g,-50% sur le 2ème
2,LAY'S Chips nature maxi format,"3,49€",370g,-50% sur le 2ème
3,OMO Lessive capsules 3en1 essences naturelles ...,"11,99€\n7,91€",29 capsules,None
4,Mangue affinée bateau,"0,95€",1 pièce,None
...,...,...,...,...
8086,AUCHAN Mozzarella en boule,"1,05€",125g,None
8087,Concombre,"1,39€",1 pièce,None
8088,Bananes Prix Bas,"0,99€",5 pièces,None
8089,AUCHAN Penne rigate,"0,94€",500g,None


In [10]:
# Sauvegarder les données dans un fichier Excel
df.to_csv("auchan_products.csv", index=False)

print("Données enregistrées dans auchan_products.csv")

Données enregistrées dans auchan_products.csv


# Scraping Carrefour

Ensuite, nous avons scrappé le site de Carrefour afin de récupérer les informations nécessaires sur les produits en promotion.

### Étapes principales :
1. Initialisation du **WebDriver** avec Selenium pour naviguer sur le site.
2. Parcours des différentes pages de produits disponibles.
3. Extraction des informations principales, telles que :
   - Nom du produit
   - Prix
   - Quantité ou format
   - Notes et avis
   - Promotions ou réductions
4. Sauvegarde des données dans un fichier CSV pour une analyse ultérieure.


In [ ]:
#Initialiser le webdriver pour firefox
driver = webdriver.Firefox()

driver.get("https://www.carrefour.fr/promotions")

try:
    reject_all = driver.find_element(By.XPATH, "//*[@id='onetrust-reject-all-handler']")
    reject_all.click()
except Exception as e:
    print(f"Erreur : {e}")



In [ ]:
import time
import csv
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# URL de base sans le numéro de page
base_url = "https://www.carrefour.fr/promotions?noRedirect=1&page="

# Listes pour stocker les informations
product_names = []
product_prices = []
quantities = []
ratings = []
reviews = []
promotions = []

page_num = 1  # Démarrer à la première page

# Boucle tant qu'il y a des pages à scraper
while True:
    print(f"Scraping page {page_num}")
    driver.get(base_url + str(page_num))  # Charger la page avec le numéro

    # Attendre un peu pour le chargement de la page
    time.sleep(3)
    
    # Récupérer les produits sur la page actuelle
    products = driver.find_elements(By.XPATH, "//article[contains(@class, 'product-card')]")
    
    if not products:
        print(f"Aucun produit trouvé sur la page {page_num}. Fin de la pagination.")
        break
    
    print(f"Nombre de produits trouvés sur la page {page_num}: {len(products)}")
    
    # Scraper les informations des produits sur cette page
    for p in products:
        # Localiser l'élément contenant le nom du produit
        try:
            name_element = p.find_element(By.XPATH, ".//h3[contains(@class, 'product-card-title__text')]")
            product_names.append(name_element.text)
        except NoSuchElementException:
            product_names.append("Nom non disponible")
        
        # Localiser l'élément contenant le prix du produit
        try:
            price_element = p.find_element(By.XPATH, ".//p[contains(@class, 'product-price__content')]")
            product_prices.append(price_element.text)
        except NoSuchElementException:
            product_prices.append("Prix non disponible")
        
        # Localiser l'élément contenant la quantité ou le format
        try:
            quantity_element = p.find_element(By.XPATH, ".//div[contains(@class, 'ds-format')]")
            quantities.append(quantity_element.text)
        except NoSuchElementException:
            quantities.append("Quantité non disponible")
        
        # Localiser l'élément contenant la note (étoiles)
        try:
            stars = p.find_element(By.XPATH, ".//span[@class='c-rating__stars']")
            full_stars = stars.find_elements(By.CLASS_NAME, "c-icon-star-fill")
            half_stars = stars.find_elements(By.CLASS_NAME, "c-icon-star-semi-fill")
            rating = len(full_stars) + 0.5 * len(half_stars)
            ratings.append(f"{rating} étoiles")
        except NoSuchElementException:
            ratings.append("Note non disponible")
        
        # Localiser l'élément contenant le nombre d'avis
        try:
            review_element = p.find_element(By.XPATH, ".//p[contains(@class, 'c-text--style-caption')]")
            reviews.append(review_element.text)
        except NoSuchElementException:
            reviews.append("Avis non disponible")
    
        # Localiser l'élément contenant la promotion ou la réduction
        try:
            promotion_element = p.find_element(By.XPATH, ".//span[contains(@class, 'promotion-label-refonte__text')]")
            promotions.append(promotion_element.text)
        except NoSuchElementException:
            promotions.append("Pas de promotion")

    # Passer à la page suivante
    page_num += 1

# Sauvegarder les informations extraites dans un fichier CSV
with open('carrefour_products.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Nom', 'Prix', 'Quantité', 'Note', 'Nombre d\'avis', 'Réduction'])
    for i in range(len(product_names)):
        writer.writerow([product_names[i], product_prices[i], quantities[i], ratings[i], reviews[i], promotions[i]])

print(f"{len(product_names)} produits ont été extraits et sauvegardés dans 'carrefour_products.csv'.")

In [ ]:
import pandas as pd 

# Charger le fichier CSV (mets le chemin de ton fichier CSV à la place de 'file_path')
file_path = 'carrefour_products.csv'
data = pd.read_csv(file_path)

# Garde uniquement les colonnes nécessaires : Nom, Prix, Quantité, Réduction
cleaned_data = data[['Nom', 'Prix', 'Quantité', 'Réduction']]

# Suppression des espaces superflus dans les noms des produits
cleaned_data['Nom'] = cleaned_data['Nom'].str.strip()

# Suppression des espaces dans la colonne Quantité et normalisation
cleaned_data['Quantité'] = cleaned_data['Quantité'].str.strip()

# Renommer la colonne 'Quantité' en 'Unité' seulement
cleaned_data = cleaned_data.rename(columns={'Quantité': 'Unité'})

# Nettoyage de la colonne Réduction (par exemple, uniformiser les valeurs dans la colonne Réduction)
cleaned_data['Réduction'] = cleaned_data['Réduction'].str.lower().str.strip()

# Affichage des premières lignes pour vérifier
print(cleaned_data.head())

# Exporter les données nettoyées si nécessaire
cleaned_data.to_csv('carrefour_product_final.csv', index=False)



In [ ]:
import pandas as pd


df = pd.DataFrame({
    'Nom': product_names,
    'Prix': product_prices,
    'Quantité': quantities,
    'Promotion': promotions,
    'Note': ratings,
    'Avis': reviews
    
})

df

In [14]:
data = pd.read_csv("C:/Users/oumis/Downloads/carrefour_product_final.xls")
data

,Nom,Prix,Unité,Réduction
0,Bonbons Assortiment Chauve qui Peut Pik Hallow...,5,700g,promo : 30%
1,Bonbons Assortiment World Mix Méga Boîte HARIBO,5,850g,prenez en 3 = payez en 2
2,Bonbons pomme fraise TETES BRULEES,2,Quantité non disponible,le 2ème à -68%
3,Bouchées Chocolat au Lait Noisettes et Gaufret...,7,375g,le 2ème à -50%
4,"Bouchées Chocolat Noir, Liqueur et Cerise MON ...",7,315g,le 2ème à -50%
...,...,...,...,...
11983,Whisky Bourbon Kentucky Straight 40% JIM BEAM,18,70cL,pas de promotion
11984,Liqueur de Whisky Honey JACK DANIEL'S,32,1L,pas de promotion
11985,Liqueur de Whisky Honey JACK DANIEL'S,22,70cL,pas de promotion
11986,Whisky Old N°7 JACK DANIEL'S,22,70cL,pas de promotion


In [16]:
# Sauvegarder les données dans un fichier csv
df.to_csv("carrefour_product_final.csv", index=False)

print("Données enregistrées dans carrefour_product_final.csv")

Données enregistrées dans carrefour_product_final.csv


## Procédure de Nettoyage des Données

Pour garantir l'uniformité et la qualité des données provenant des deux bases (Auchan et Carrefour), nous avons appliqué les étapes suivantes :

### Étapes principales de nettoyage :
1. **Renommer les colonnes** : 
   - Standardisation des noms de colonnes (`nom`, `prix`, `unite`, `reduction`) pour une manipulation facile et cohérente.

2. **Suppression des doublons** : 
   - Éliminer les lignes en double pour éviter les redondances dans les données.

3. **Nettoyage de la colonne `prix`** : 
   - Gestion des prix multiples en sélectionnant le prix final (par exemple, en cas de promotion).
   - Suppression des caractères inutiles comme le symbole `€` et remplacement des virgules par des points pour assurer un format numérique.

4. **Standardisation de la colonne `unite`** : 
   - Conversion explicite des unités :
     - `cl` converti en `ml`.
     - `kg` converti en `g`.
     - `l` converti en `ml`.
   - Uniformisation de la casse et suppression des espaces inutiles.
   - Gestion des cas particuliers comme les `capsules`, `sachets`, etc., sans modification.

5. **Gestion des valeurs manquantes** : 
   - Remplir les valeurs manquantes dans la colonne `reduction` par `"Aucune réduction"`.
   - Remplir les valeurs manquantes dans la colonne `unite` par `"quantité inconnue"`.

6. **Sauvegarde des données nettoyées** : 
   - Les données nettoyées sont sauvegardées dans un fichier CSV distinct (`auchan_cleaned.csv` ou équivalent pour Carrefour).

### Objectifs du nettoyage :
- Uniformiser les formats de données pour permettre une comparaison directe entre les bases de données des deux enseignes.
- S'assurer que toutes les valeurs sont lisibles et cohérentes pour une analyse statistique et visuelle ultérieure.

Ce processus garantit des données propres, cohérentes et prêtes pour une analyse comparative entre Auchan et Carrefour.


### AUCHAN

In [33]:
import pandas as pd

# Charger la base Auchan
auchan_file_path = 'Auchan_products.csv'
auchan_data = pd.read_csv(auchan_file_path)

# 1. Renommer les colonnes pour une standardisation
auchan_data.columns = ['nom', 'prix', 'unite', 'reduction']

# 2. Supprimer les doublons
auchan_data = auchan_data.drop_duplicates()

# 3. Nettoyer la colonne 'prix'
def clean_price(price):
    """
    Nettoie la colonne des prix :
    - Sélectionne le second prix si plusieurs sont indiqués (promo).
    - Supprime les caractères inutiles comme € et les espaces.
    """
    if isinstance(price, str):
        parts = price.split("\n")  # Sépare les prix si plusieurs sont présents
        selected_price = parts[-1] if len(parts) > 1 else parts[0]  # Prend le second prix s'il existe
        return selected_price.replace("€", "").replace(",", ".").strip()
    return price

auchan_data['prix'] = auchan_data['prix'].apply(clean_price)

# 4. Nettoyer et standardiser la colonne 'unite'
def standardize_unit(unit):
    """
    Standardise les unités :
    - Convertit tout en minuscules.
    - Convertit explicitement "cl" en "ml".
    - Conserve les autres unités explicites comme "29 capsules".
    """
    if isinstance(unit, str):
        unit = unit.strip().lower()
        # Remplacer les virgules par des points
        unit = unit.replace(",", ".")
        # Gestion explicite des centilitres (cl) → millilitres (ml)
        if "cl" in unit:
            try:
                value = float(unit.replace("cl", "").strip()) * 10
                return f"{value}ml"
            except ValueError:
                return "quantité inconnue"  # Marquer comme inconnue si conversion échoue
        # Conversion standard pour kg → g
        if "kg" in unit:
            try:
                value = float(unit.replace("kg", "").strip()) * 1000
                return f"{value}g"
            except ValueError:
                return unit
        # Conversion standard pour l → ml
        if "l" in unit:
            try:
                value = float(unit.replace("l", "").strip()) * 1000
                return f"{value}ml"
            except ValueError:
                return unit
        # Gestion des grammes et millilitres déjà standardisés
        if unit.endswith("g") or unit.endswith("ml"):
            return unit
        # Conserve les unités explicites (ex: capsules, sachets, etc.)
        return unit
    return "quantité inconnue"

auchan_data['unite'] = auchan_data['unite'].apply(standardize_unit)

# 5. Gérer les valeurs manquantes
auchan_data['reduction'] = auchan_data['reduction'].fillna("Aucune réduction")  # Remplit les valeurs manquantes dans la colonne 'reduction'
auchan_data['unite'] = auchan_data['unite'].fillna("quantité inconnue")  # Remplit les valeurs manquantes dans la colonne 'unite'

# 6. Sauvegarder les données nettoyées
auchan_cleaned_file = 'auchan_cleaned.csv' 
auchan_data.to_csv(auchan_cleaned_file, index=False)

# Nettoyage terminé pour Auchan
print("Nettoyage terminé pour Auchan. Données sauvegardées sous forme de fichier CSV nettoyé.")

Nettoyage terminé pour Auchan. Données sauvegardées sous forme de fichier CSV nettoyé.


### CARREFOUR

In [45]:
import pandas as pd

# Charger la base Auchan
carrefour_file_path = 'carrefour_product_final.csv'
carrefour_data = pd.read_csv(carrefour_file_path)

# 1. Renommer les colonnes pour une standardisation
carrefour_data.columns = ['nom', 'prix', 'unite', 'reduction']

# 2. Supprimer les doublons
carrefour_data = carrefour_data.drop_duplicates()

# 3. Nettoyer la colonne 'prix'
def clean_price(price):
    """
    Nettoie la colonne des prix :
    - Sélectionne le second prix si plusieurs sont indiqués (promo).
    - Supprime les caractères inutiles comme € et les espaces.
    """
    if isinstance(price, str):
        parts = price.split("\n")  # Sépare les prix si plusieurs sont présents
        selected_price = parts[-1] if len(parts) > 1 else parts[0]  # Prend le second prix s'il existe
        return selected_price.replace("€", "").replace(",", ".").strip()
    return price

carrefour_data['prix'] = carrefour_data['prix'].apply(clean_price)

# 4. Nettoyer et standardiser la colonne 'unite'
def standardize_unit(unit):
    """
    Standardise les unités :
    - Convertit tout en minuscules.
    - Convertit explicitement "cl" en "ml".
    - Conserve les autres unités explicites comme "29 capsules".
    """
    if isinstance(unit, str):
        unit = unit.strip().lower()
        # Remplacer les virgules par des points
        unit = unit.replace(",", ".")
        # Gestion explicite des centilitres (cl) → millilitres (ml)
        if "cl" in unit:
            try:
                value = float(unit.replace("cl", "").strip()) * 10
                return f"{value}ml"
            except ValueError:
                return "quantité inconnue"  # Marquer comme inconnue si conversion échoue
        # Conversion standard pour kg → g
        if "kg" in unit:
            try:
                value = float(unit.replace("kg", "").strip()) * 1000
                return f"{value}g"
            except ValueError:
                return unit
        # Conversion standard pour l → ml
        if "l" in unit:
            try:
                value = float(unit.replace("l", "").strip()) * 1000
                return f"{value}ml"
            except ValueError:
                return unit
        # Gestion des grammes et millilitres déjà standardisés
        if unit.endswith("g") or unit.endswith("ml"):
            return unit
        # Conserve les unités explicites (ex: capsules, sachets, etc.)
        return unit
    return "quantité inconnue"

carrefour_data['unite'] = carrefour_data['unite'].apply(standardize_unit)

# 5. Gérer les valeurs manquantes
carrefour_data['reduction'] = carrefour_data['reduction'].fillna("Aucune réduction")  # Remplit les valeurs manquantes dans la colonne 'reduction'
carrefour_data['unite'] = carrefour_data['unite'].fillna("quantité inconnue")  # Remplit les valeurs manquantes dans la colonne 'unite'

# 6. Sauvegarder les données nettoyées
carrefour_cleaned_file = 'carrefour_cleaned.csv' 
carrefour_data.to_csv(carrefour_cleaned_file, index=False)

# Nettoyage terminé pour Auchan
print("Nettoyage terminé pour carrefour. Données sauvegardées sous forme de fichier CSV nettoyé.")

Nettoyage terminé pour carrefour. Données sauvegardées sous forme de fichier CSV nettoyé.


## Scraping des recettes Marmiton

Nous avons réalisé un scraping du site **Marmiton** pour collecter les recettes des différentes catégories suivantes : **entrées**, **plats**, **desserts**, et **apéritifs**.

### Étapes principales du scraping :

1. **Initialisation du navigateur :**
   - Un **WebDriver Selenium** (Firefox) a été configuré en mode sans tête pour parcourir automatiquement les pages du site Marmiton.
   - Le script gère également l'acceptation des cookies sur la page d'accueil.

2. **Scraping des pages de recettes :**
   - Les URLs de chaque catégorie ont été parcourues page par page pour collecter toutes les recettes disponibles.
   - Chaque recette sur une page a été identifiée et son URL a été sauvegardée pour extraction détaillée.

3. **Extraction des détails de chaque recette :**
   - Les informations suivantes ont été extraites pour chaque recette :
     - **Nom de la recette** : Identifié à partir du titre de la recette.
     - **Ingrédients** : Quantités, unités, noms des ingrédients et leurs éventuels compléments.
   - Une gestion a été mise en place pour cliquer sur le bouton "Voir plus" lorsque la liste des ingrédients était tronquée.

4. **Gestion des erreurs et des interruptions :**
   - Des mécanismes de gestion des exceptions permettent de continuer le scraping même en cas de problème avec certaines pages ou recettes.
   - Le navigateur est redémarré toutes les 5 pages pour libérer la mémoire et garantir la stabilité du processus.

5. **Sauvegarde des données :**
   - Les données collectées (nom de la recette, URL, et ingrédients) ont été enregistrées dans un fichier **CSV** nommé `marmiton_recipes.csv`.

### Catégories ciblées :
- **Entrées**
- **Plats**
- **Desserts**
- **Apéritifs**

Ces étapes permettent d'obtenir une base de données structurée contenant des informations détaillées sur un large éventail de recettes issues de Marmiton, qui pourront être utilisées pour des analyses ou des comparaisons avec les produits disponibles en supermarché.


### Scrapping des entrées

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import gc

# Initialisation du webdriver pour ouvrir le site de Marmiton
def start_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')  # Mode sans tête pour économiser la mémoire
    driver = webdriver.Firefox(options=options)
    return driver

driver = start_driver()


In [ ]:
# Accepter les cookies
def accept_cookies(driver):
    try:
        driver.get("https://www.marmiton.org/")
        time.sleep(5)  # Attendre que la page charge complètement
        accept_cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]/span")
        accept_cookies_button.click()
        print("Bouton 'J'accepte tout' cliqué avec succès !")
    except Exception as e:
        print(f"Erreur lors du clic sur 'J'accepte tout' : {e}")

accept_cookies(driver)

# Fonction pour scraper une page
def scrape_page(driver, url):
    recipes = []
    try:
        driver.get(url)
        time.sleep(5)  # Attendre que la page charge complètement

        # Vérifier que la page est correctement chargée
        if "recettes" not in driver.current_url:
            print(f"La page {url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return recipes

        # Trouver tous les éléments contenant les recettes
        recipe_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div/div/div/a')
        print(f"{len(recipe_elements)} recettes trouvées sur {url}")

        for recipe in recipe_elements:
            try:
                # Extraire l'URL de la recette
                recipe_url = recipe.get_attribute("href")
                recipes.append({"url": recipe_url})
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une recette : {e}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {e}")
    return recipes

# Fonction pour scraper les ingrédients d'une recette
def scrape_recipe_details(driver, recipe_url):
    try:
        driver.get(recipe_url)
        print(f"Scraping les détails pour : {recipe_url}")

        # Vérifier que la page de recette est correctement chargée
        if "recette" not in driver.current_url:
            print(f"La recette {recipe_url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return "", ""

        # Extraire le nom de la recette
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h1'))
            ).text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du titre : {e}")
            title = ""

        # Cliquer sur "Voir plus" pour charger tous les ingrédients
        try:
            while True:
                voir_plus_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[3]/span[1]'))
                )
                voir_plus_button.click()
                time.sleep(2)
        except:
            pass

        # Extraire les ingrédients
        ingredients = []
        try:
            ingredient_blocks = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[2]/div/div'))
            )
            for block in ingredient_blocks:
                try:
                    quantity = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]') else ""
                    unit = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]') else ""
                    name = block.find_element(By.XPATH, './/span[@class="ingredient-name"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-name"]') else ""
                    complement = block.find_element(By.XPATH, './/span[@class="ingredient-complement"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-complement"]') else ""
                    ingredient = f"{quantity} {unit} {name} {complement}".strip()
                    ingredients.append(ingredient)
                except Exception as e:
                    print(f"Erreur lors de l'extraction d'un ingrédient : {e}")
        except Exception as e:
            print(f"Erreur lors de l'extraction des ingrédients : {e}")

        # Afficher les données extraites pour validation
        print("Données extraites :")
        print(f"Nom : {title}")
        print(f"Ingrédients : {', '.join(ingredients)}")

        # Si aucune donnée n'est extraite, retour immédiat
        if not title and not ingredients:
            print("Aucune donnée extraite pour cette recette, arrêt du processus.")
            return "", ""

        return title, ", ".join(ingredients)

    except Exception as e:
        print(f"Erreur lors du scraping des détails pour {recipe_url} : {e}")
        return "", ""

# Fonction principale pour le scraping
def main():
    global driver

    # URL de base pour les pages "Entrées"
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=entr%C3%A9es&page="
    total_pages = 67  # Nombre total de pages à scraper

    # Créer un fichier CSV
    fieldnames = ["Title", "URL", "Ingredients"]
    with open("marmiton_recipes.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        unique_urls = set()

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}...")
            page_url = f"{base_url}{page}"
            recipes = scrape_page(driver, page_url)

            for recipe in recipes:
                if recipe["url"] not in unique_urls:
                    unique_urls.add(recipe["url"])
                    title, ingredients = scrape_recipe_details(driver, recipe["url"])
                    writer.writerow({"Title": title, "URL": recipe["url"], "Ingredients": ingredients})

            print(f"Page {page} terminée. Total de recettes uniques extraites jusqu'à présent : {len(unique_urls)}")

            # Redémarrer le WebDriver tous les 5 pages pour éviter les erreurs de mémoire
            if page % 5 == 0:
                driver.quit()
                print("Redémarrage du navigateur pour libérer la mémoire...")
                driver = start_driver()
                accept_cookies(driver)

            time.sleep(5)  # Pause pour éviter d'être bloqué

    driver.quit()
    print("Scraping terminé. Résultats enregistrés dans 'marmiton_recipes.csv'.")

if __name__ == "__main__":
    main()


### Scrapping des plats

In [ ]:
# Accepter les cookies
def accept_cookies(driver):
    try:
        driver.get("https://www.marmiton.org/")
        time.sleep(5)  # Attendre que la page charge complètement
        accept_cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]/span")
        accept_cookies_button.click()
        print("Bouton 'J'accepte tout' cliqué avec succès !")
    except Exception as e:
        print(f"Erreur lors du clic sur 'J'accepte tout' : {e}")

accept_cookies(driver)

# Fonction pour scraper une page
def scrape_page(driver, url):
    recipes = []
    try:
        driver.get(url)
        time.sleep(5)  # Attendre que la page charge complètement

        # Vérifier que la page est correctement chargée
        if "recettes" not in driver.current_url:
            print(f"La page {url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return recipes

        # Trouver tous les éléments contenant les recettes
        recipe_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div/div/div/a')
        print(f"{len(recipe_elements)} recettes trouvées sur {url}")

        for recipe in recipe_elements:
            try:
                # Extraire l'URL de la recette
                recipe_url = recipe.get_attribute("href")
                recipes.append({"url": recipe_url})
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une recette : {e}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {e}")
    return recipes

# Fonction pour scraper les ingrédients d'une recette
def scrape_recipe_details(driver, recipe_url):
    try:
        driver.get(recipe_url)
        print(f"Scraping les détails pour : {recipe_url}")

        # Vérifier que la page de recette est correctement chargée
        if "recette" not in driver.current_url:
            print(f"La recette {recipe_url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return "", ""

        # Extraire le nom de la recette
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h1'))
            ).text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du titre : {e}")
            title = ""

        # Cliquer sur "Voir plus" pour charger tous les ingrédients
        try:
            while True:
                voir_plus_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[3]/span[1]'))
                )
                voir_plus_button.click()
                time.sleep(2)
        except:
            pass

        # Extraire les ingrédients
        ingredients = []
        try:
            ingredient_blocks = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[2]/div/div'))
            )
            for block in ingredient_blocks:
                try:
                    quantity = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]') else ""
                    unit = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]') else ""
                    name = block.find_element(By.XPATH, './/span[@class="ingredient-name"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-name"]') else ""
                    complement = block.find_element(By.XPATH, './/span[@class="ingredient-complement"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-complement"]') else ""
                    ingredient = f"{quantity} {unit} {name} {complement}".strip()
                    ingredients.append(ingredient)
                except Exception as e:
                    print(f"Erreur lors de l'extraction d'un ingrédient : {e}")
        except Exception as e:
            print(f"Erreur lors de l'extraction des ingrédients : {e}")

        # Afficher les données extraites pour validation
        print("Données extraites :")
        print(f"Nom : {title}")
        print(f"Ingrédients : {', '.join(ingredients)}")

        # Si aucune donnée n'est extraite, retour immédiat
        if not title and not ingredients:
            print("Aucune donnée extraite pour cette recette, arrêt du processus.")
            return "", ""

        return title, ", ".join(ingredients)

    except Exception as e:
        print(f"Erreur lors du scraping des détails pour {recipe_url} : {e}")
        return "", ""

# Fonction principale pour le scraping
def main():
    global driver

    # URL de base pour les pages "Entrées"
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=plats&page="
    total_pages = 67  # Nombre total de pages à scraper

    # Créer un fichier CSV
    fieldnames = ["Title", "URL", "Ingredients"]
    with open("marmiton_recipes_plats.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        unique_urls = set()

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}...")
            page_url = f"{base_url}{page}"
            recipes = scrape_page(driver, page_url)

            for recipe in recipes:
                if recipe["url"] not in unique_urls:
                    unique_urls.add(recipe["url"])
                    title, ingredients = scrape_recipe_details(driver, recipe["url"])
                    writer.writerow({"Title": title, "URL": recipe["url"], "Ingredients": ingredients})

            print(f"Page {page} terminée. Total de recettes uniques extraites jusqu'à présent : {len(unique_urls)}")

            # Redémarrer le WebDriver tous les 5 pages pour éviter les erreurs de mémoire
            if page % 5 == 0:
                driver.quit()
                print("Redémarrage du navigateur pour libérer la mémoire...")
                driver = start_driver()
                accept_cookies(driver)

            time.sleep(5)  # Pause pour éviter d'être bloqué

    driver.quit()
    print("Scraping terminé. Résultats enregistrés dans 'marmiton_recipes_plats.csv'.")

if __name__ == "__main__":
    main()


### Scrapping des desserts

In [ ]:
# Accepter les cookies
def accept_cookies(driver):
    try:
        driver.get("https://www.marmiton.org/")
        time.sleep(5)  # Attendre que la page charge complètement
        accept_cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]/span")
        accept_cookies_button.click()
        print("Bouton 'J'accepte tout' cliqué avec succès !")
    except Exception as e:
        print(f"Erreur lors du clic sur 'J'accepte tout' : {e}")

accept_cookies(driver)

# Fonction pour scraper une page
def scrape_page(driver, url):
    recipes = []
    try:
        driver.get(url)
        time.sleep(5)  # Attendre que la page charge complètement

        # Vérifier que la page est correctement chargée
        if "recettes" not in driver.current_url:
            print(f"La page {url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return recipes

        # Trouver tous les éléments contenant les recettes
        recipe_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div/div/div/a')
        print(f"{len(recipe_elements)} recettes trouvées sur {url}")

        for recipe in recipe_elements:
            try:
                # Extraire l'URL de la recette
                recipe_url = recipe.get_attribute("href")
                recipes.append({"url": recipe_url})
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une recette : {e}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {e}")
    return recipes

# Fonction pour scraper les ingrédients d'une recette
def scrape_recipe_details(driver, recipe_url):
    try:
        driver.get(recipe_url)
        print(f"Scraping les détails pour : {recipe_url}")

        # Vérifier que la page de recette est correctement chargée
        if "recette" not in driver.current_url:
            print(f"La recette {recipe_url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return "", ""

        # Extraire le nom de la recette
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h1'))
            ).text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du titre : {e}")
            title = ""

        # Cliquer sur "Voir plus" pour charger tous les ingrédients
        try:
            while True:
                voir_plus_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[3]/span[1]'))
                )
                voir_plus_button.click()
                time.sleep(2)
        except:
            pass

        # Extraire les ingrédients
        ingredients = []
        try:
            ingredient_blocks = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[2]/div/div'))
            )
            for block in ingredient_blocks:
                try:
                    quantity = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]') else ""
                    unit = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]') else ""
                    name = block.find_element(By.XPATH, './/span[@class="ingredient-name"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-name"]') else ""
                    complement = block.find_element(By.XPATH, './/span[@class="ingredient-complement"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-complement"]') else ""
                    ingredient = f"{quantity} {unit} {name} {complement}".strip()
                    ingredients.append(ingredient)
                except Exception as e:
                    print(f"Erreur lors de l'extraction d'un ingrédient : {e}")
        except Exception as e:
            print(f"Erreur lors de l'extraction des ingrédients : {e}")

        # Afficher les données extraites pour validation
        print("Données extraites :")
        print(f"Nom : {title}")
        print(f"Ingrédients : {', '.join(ingredients)}")

        # Si aucune donnée n'est extraite, retour immédiat
        if not title and not ingredients:
            print("Aucune donnée extraite pour cette recette, arrêt du processus.")
            return "", ""

        return title, ", ".join(ingredients)

    except Exception as e:
        print(f"Erreur lors du scraping des détails pour {recipe_url} : {e}")
        return "", ""

# Fonction principale pour le scraping
def main():
    global driver

    # URL de base pour les pages "Desserts"
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=desserts&page="
    total_pages = 67  # Nombre total de pages à scraper

    # Créer un fichier CSV
    fieldnames = ["Title", "URL", "Ingredients"]
    with open("marmiton_desserts.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        unique_urls = set()

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}...")
            page_url = f"{base_url}{page}"
            recipes = scrape_page(driver, page_url)

            for recipe in recipes:
                if recipe["url"] not in unique_urls:
                    unique_urls.add(recipe["url"])
                    title, ingredients = scrape_recipe_details(driver, recipe["url"])
                    writer.writerow({"Title": title, "URL": recipe["url"], "Ingredients": ingredients})

            print(f"Page {page} terminée. Total de recettes uniques extraites jusqu'à présent : {len(unique_urls)}")

            # Redémarrer le WebDriver tous les 5 pages pour éviter les erreurs de mémoire
            if page % 5 == 0:
                driver.quit()
                print("Redémarrage du navigateur pour libérer la mémoire...")
                driver = start_driver()
                accept_cookies(driver)

            time.sleep(5)  # Pause pour éviter d'être bloqué

    driver.quit()
    print("Scraping terminé. Résultats enregistrés dans 'marmiton_desserts.csv'.")

if __name__ == "__main__":
    main()


### Scrapping des apéritifs

In [ ]:
# Accepter les cookies
def accept_cookies(driver):
    try:
        driver.get("https://www.marmiton.org/")
        time.sleep(5)  # Attendre que la page charge complètement
        accept_cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]/span")
        accept_cookies_button.click()
        print("Bouton 'J'accepte tout' cliqué avec succès !")
    except Exception as e:
        print(f"Erreur lors du clic sur 'J'accepte tout' : {e}")

accept_cookies(driver)

# Fonction pour scraper une page
def scrape_page(driver, url):
    recipes = []
    try:
        driver.get(url)
        time.sleep(5)  # Attendre que la page charge complètement

        # Vérifier que la page est correctement chargée
        if "recettes" not in driver.current_url:
            print(f"La page {url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return recipes

        # Trouver tous les éléments contenant les recettes
        recipe_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div/div/div/a')
        print(f"{len(recipe_elements)} recettes trouvées sur {url}")

        for recipe in recipe_elements:
            try:
                # Extraire l'URL de la recette
                recipe_url = recipe.get_attribute("href")
                recipes.append({"url": recipe_url})
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une recette : {e}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {e}")
    return recipes

# Fonction pour scraper les ingrédients d'une recette
def scrape_recipe_details(driver, recipe_url):
    try:
        driver.get(recipe_url)
        print(f"Scraping les détails pour : {recipe_url}")

        # Vérifier que la page de recette est correctement chargée
        if "recette" not in driver.current_url:
            print(f"La recette {recipe_url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return "", ""

        # Extraire le nom de la recette
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h1'))
            ).text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du titre : {e}")
            title = ""

        # Cliquer sur "Voir plus" pour charger tous les ingrédients
        try:
            while True:
                voir_plus_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[3]/span[1]'))
                )
                voir_plus_button.click()
                time.sleep(2)
        except:
            pass

        # Extraire les ingrédients
        ingredients = []
        try:
            ingredient_blocks = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[2]/div/div'))
            )
            for block in ingredient_blocks:
                try:
                    quantity = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]') else ""
                    unit = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]') else ""
                    name = block.find_element(By.XPATH, './/span[@class="ingredient-name"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-name"]') else ""
                    complement = block.find_element(By.XPATH, './/span[@class="ingredient-complement"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-complement"]') else ""
                    ingredient = f"{quantity} {unit} {name} {complement}".strip()
                    ingredients.append(ingredient)
                except Exception as e:
                    print(f"Erreur lors de l'extraction d'un ingrédient : {e}")
        except Exception as e:
            print(f"Erreur lors de l'extraction des ingrédients : {e}")

        # Afficher les données extraites pour validation
        print("Données extraites :")
        print(f"Nom : {title}")
        print(f"Ingrédients : {', '.join(ingredients)}")

        # Si aucune donnée n'est extraite, retour immédiat
        if not title and not ingredients:
            print("Aucune donnée extraite pour cette recette, arrêt du processus.")
            return "", ""

        return title, ", ".join(ingredients)

    except Exception as e:
        print(f"Erreur lors du scraping des détails pour {recipe_url} : {e}")
        return "", ""

# Fonction principale pour le scraping
def main():
    global driver

    # URL de base pour les pages "Apéritifs"
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=ap%C3%A9ritifs&page="
    total_pages = 24  # Nombre total de pages à scraper

    # Créer un fichier CSV
    fieldnames = ["Title", "URL", "Ingredients"]
    with open("marmiton_apéritifs.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        unique_urls = set()

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}...")
            page_url = f"{base_url}{page}"
            recipes = scrape_page(driver, page_url)

            for recipe in recipes:
                if recipe["url"] not in unique_urls:
                    unique_urls.add(recipe["url"])
                    title, ingredients = scrape_recipe_details(driver, recipe["url"])
                    writer.writerow({"Title": title, "URL": recipe["url"], "Ingredients": ingredients})

            print(f"Page {page} terminée. Total de recettes uniques extraites jusqu'à présent : {len(unique_urls)}")

            # Redémarrer le WebDriver tous les 5 pages pour éviter les erreurs de mémoire
            if page % 5 == 0:
                driver.quit()
                print("Redémarrage du navigateur pour libérer la mémoire...")
                driver = start_driver()
                accept_cookies(driver)

            time.sleep(5)  # Pause pour éviter d'être bloqué

    driver.quit()
    print("Scraping terminé. Résultats enregistrés dans 'marmiton_apéritifs.csv'.")

if __name__ == "__main__":
    main()


## Combinaison et nettoyage des bases de données des recettes

### Étape 1 : Fusion des bases de données
Nous avons fusionné les différentes bases de données obtenues lors du scraping de Marmiton :
- **Entrées** (`marmiton_recipes.csv`)
- **Plats** (`marmiton_recipes_plats.csv`)
- **Desserts** (`marmiton_desserts.csv`)
- **Apéritifs** (`marmiton_apéritifs.csv`)

La fusion a été réalisée à l'aide de la méthode `pd.concat()` de **pandas**, qui permet de combiner les données tout en réinitialisant les index.

Une fois les bases combinées, les lignes contenant des valeurs manquantes ont été supprimées pour garantir la qualité des données.

Le fichier résultant a été enregistré sous le nom `base_recettes.csv`.




In [ ]:
import pandas as pd

# Charger les fichiers CSV dans des DataFrames
file1= 'marmiton_recipes.csv'
file2= 'marmiton_recipes_plats.csv'
file3= 'marmiton_desserts.csv'
file4= 'marmiton_apéritifs.csv'

df1= pd.read_csv(file1)
df2= pd.read_csv(file2)
df3= pd.read_csv(file3)
df4= pd.read_csv(file4)

# Fusionner les deux DataFrames
# `ignore_index=True` réinitialise les index après la fusion
base_recettes= pd.concat([df1, df2, df3, df4], ignore_index=True)

# Supprimer les lignes avec des valeurs manquantes
base_recettes = base_recettes.dropna()

# Enregistrer le DataFrame fusionné dans un nouveau fichier CSV
output_file = "base_recettes.csv"
base_recettes.to_csv(output_file, index=False)

print(f"La fusion est terminée. Le fichier fusionné est enregistré sous le nom : {output_file}")


                


### Étape 2 : Nettoyage initial des données fusionnées
- **Suppression des lignes non valides** : Les lignes dont la colonne `Ingredients` contenait uniquement des virgules ou des espaces ont été supprimées.
- **Renommage des colonnes** : La colonne `Title` a été renommée en `recipe_name` pour une meilleure lisibilité.

Le fichier nettoyé a été sauvegardé sous le nom `base_recettes_cleaned.csv`.



In [43]:
import pandas as pd

# Charger la base de données des recettes
base_recettes_file = "base_recettes.csv"  # Remplacez par le chemin de votre fichier
base_recettes = pd.read_csv(base_recettes_file)

# Supprimer les lignes où la colonne 'Ingredients' contient uniquement des virgules (ou des espaces)
base_recettes_cleaned = base_recettes[~base_recettes['Ingredients'].str.fullmatch(r'[,\s]*')]

# Renommer la colonne 'Title' en 'name_recipe'
base_recettes_cleaned = base_recettes_cleaned.rename(columns={'Title': 'recipe_name'})

# Sauvegarder la base de données nettoyée
base_recettes_cleaned.to_csv("base_recettes_cleaned.csv", index=False)

print("Nettoyage terminé. Les lignes contenant uniquement des virgules ont été supprimées.")


Nettoyage terminé. Les lignes contenant uniquement des virgules ont été supprimées.


### Étape 3 : Nettoyage avancé des ingrédients
Pour rendre les données exploitables, les ingrédients ont été standardisés grâce à une fonction de nettoyage dédiée :
- **Mise en minuscule** : Tous les textes ont été convertis en minuscules.
- **Suppression des quantités et unités de mesure** : Les unités telles que `kg`, `g`, `litres`, ou les expressions comme `cuillère à soupe` ont été supprimées.
- **Élimination des déterminants et mots de liaison** : Les mots comme `de`, `du`, `la`, `et`, etc., ont été enlevés.
- **Suppression des adjectifs redondants** : Des termes descriptifs inutiles, tels que `petit`, `gros`, `rouge`, ont été supprimés pour simplifier les noms des ingrédients.
- **Nettoyage des espaces** : Les espaces multiples ont été réduits à un espace unique.

Les ingrédients nettoyés ont été ajoutés dans une nouvelle colonne intitulée `ingredient_cleaned`.

Le fichier final a été sauvegardé sous le nom `base_recettes_cleaned_general.csv`.


### Résultat
Les données combinées et nettoyées permettent une meilleure exploitation pour des analyses ou des comparaisons, notamment avec les produits des supermarchés.

In [71]:
import pandas as pd
import re

# Charger le fichier CSV
file = pd.read_csv('base_recettes_cleaned.csv')
base_recettes_cleaned = pd.DataFrame(file)

# Fonction de nettoyage
def clean_ingredient_general(text):
    # Vérifier si le texte est une chaîne valide
    if not isinstance(text, str):
        return ""

    # Mettre en minuscule
    text = text.lower()

    # Supprimer les quantités et unités de mesure (numérique suivi de mot)
    text = re.sub(r'\b\d+\s*(kg|g|grammes|litres|l|cl|ml|cuillere|cuilleres|soupe|café|tasse|tasses)\b', '', text)

    # Supprimer les déterminants et mots de liaison courants
    text = re.sub(r'\b(un|une|des|du|de|la|le|les|et|en|à|a|pour|avec|au|aux|sur|dans|par)\b', '', text)

    # Supprimer les adjectifs et mots redondants génériques (ex: gros, petit, rouge, etc.)
    text = re.sub(r'\b(\w*ment|\w*eux|épais|épaisses|fraîche|rouge|gros|petit|grande|grands|moelleux|fondant)\b', '', text)

    # Supprimer les espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Appliquer la fonction pour créer une nouvelle colonne
base_recettes_cleaned['ingredient_cleaned'] = base_recettes_cleaned['Ingredients'].apply(clean_ingredient_general)

# Sauvegarder les données nettoyées dans un nouveau fichier CSV
output_file = 'base_recettes_cleaned_general.csv'
base_recettes_cleaned.to_csv(output_file, index=False)

print(f"Fichier nettoyé sauvegardé sous le nom : {output_file}")


Fichier nettoyé sauvegardé sous le nom : base_recettes_cleaned_general.csv


In [106]:
recipes_file = "base_recettes_cleaned_general.csv"
recipes_data = pd.read_csv(recipes_file)

print(recipes_data.head())


                                         recipe_name  \
0                 Bonhomme de neige : entrée de Noël   
1                                   Entrée fraîcheur   
2            P'tite entrée sympathique aux rillettes   
3                Entrée chaude à la bisque de homard   
4  Asperges à l'hollandaise (pour brunch chic ou ...   

                                                 URL  \
0  https://www.marmiton.org/recettes/recette_bonh...   
1  https://www.marmiton.org/recettes/recette_entr...   
2  https://www.marmiton.org/recettes/recette_p-ti...   
3  https://www.marmiton.org/recettes/recette_entr...   
4  https://www.marmiton.org/recettes/recette_aspe...   

                                         Ingredients  \
0  persil plat, 1 cuillère à café vinaigre, 1 boc...   
1  3  tomates, , 200 g chèvre frais, , 4 cuillère...   
2  gruyère râpé ., 20 cl crème fraîche, , 5  oeuf...   
3  3 cuillères à soupe maïzena, 1 verre crème fra...   
4  8  asperges fraîches (en boîte, si l'on est

# Le Modèle

### A ce stade, nous avons choisi un modéle tiré de hugging face pour calculer la similarité entre les produits et faire la correspondance entre les deux magasins

### Installation des packages 

In [86]:
import pandas as pd
import re
import unidecode
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import ttk

### Prétraitement des données et normalisation des produits

In [87]:
# Prétraitement des textes
def preprocess_text(text):
    text = unidecode.unidecode(text)  # Enlever les accents
    text = text.lower()  # Mettre en minuscule
    text = re.sub(r'\s+', ' ', text)  # Retirer les espaces multiples
    text = re.sub(r'[^a-z0-9 ]', '', text)  # Retirer les caractères spéciaux
    return text.strip()

# Charger les données
recipes_file = "base_recettes_cleaned_general.csv"
auchan_file = "auchan_cleaned.csv"
carrefour_file = "carrefour_cleaned.csv"

recipes_data = pd.read_csv(recipes_file)
auchan_data = pd.read_csv(auchan_file)
carrefour_data = pd.read_csv(carrefour_file)

# Normaliser les noms des produits
recipes_data['normalized_name'] = recipes_data['recipe_name'].apply(preprocess_text)
auchan_data['normalized_name'] = auchan_data['nom'].apply(preprocess_text)
carrefour_data['normalized_name'] = carrefour_data['nom'].apply(preprocess_text)


### Correspondance des produits

In [88]:
# Charger le modèle
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Calculer les embeddings pour les noms des produits
auchan_embeddings = model.encode(auchan_data['normalized_name'].tolist(), convert_to_tensor=True)
carrefour_embeddings = model.encode(carrefour_data['normalized_name'].tolist(), convert_to_tensor=True)

# Prétraiter un ingrédient
def preprocess_ingredient(ingredient):
    ingredient = preprocess_text(ingredient)
    ingredient = re.sub(r'\d+ ?(g|kg|ml|l|cuillère[s]?|soupe|pincée[s]?|à soupe|c. à soupe|pâte[s]?)', '', ingredient)
    return ingredient.strip()

# Recherche des correspondances pour un ingrédient
def match_ingredient(ingredient):
    # Prétraiter l'ingrédient
    ingredient = preprocess_ingredient(ingredient)
    ingredient_embedding = model.encode([ingredient], convert_to_tensor=True)

    # Correspondances Auchan
    auchan_similarities = cosine_similarity(ingredient_embedding, auchan_embeddings)[0]
    auchan_top_index = auchan_similarities.argsort()[-3:][::-1]  # Top 3
    auchan_matches = auchan_data.iloc[auchan_top_index][['nom', 'prix', 'unite']].to_dict('records')
    
    if not auchan_matches:
        auchan_matches = [{'nom': 'Aucun produit trouvé', 'prix': '-', 'unite': '-'}]

    # Correspondances Carrefour
    carrefour_similarities = cosine_similarity(ingredient_embedding, carrefour_embeddings)[0]
    carrefour_top_index = carrefour_similarities.argsort()[-3:][::-1]  # Top 3
    carrefour_matches = carrefour_data.iloc[carrefour_top_index][['nom', 'prix', 'unite']].to_dict('records')
    
    if not carrefour_matches:
        carrefour_matches = [{'nom': 'Aucun produit trouvé', 'prix': '-', 'unite': '-'}]

    return {
        'ingredient': ingredient,
        'auchan': auchan_matches,
        'carrefour': carrefour_matches
    }

# Rechercher les produits correspondant aux ingrédients
def search_products(ingredients):
    results = []
    for ingredient in ingredients:
        result = match_ingredient(ingredient)
        results.append(result)
    return results

# Fonction de recherche par nom de recette
def search_by_recipe_name():
    recipe_name = recipe_entry.get()
    normalized_name = preprocess_text(recipe_name)

    # Rechercher la recette correspondante
    matching_recipes = recipes_data[recipes_data['normalized_name'].str.contains(normalized_name)]
    if matching_recipes.empty:
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, f"Aucune recette trouvée pour '{recipe_name}'.")
        return

    # Extraire les ingrédients de la première recette trouvée
    selected_recipe = matching_recipes.iloc[0]
    ingredients = selected_recipe['ingredient_cleaned'].split(',')

    # Rechercher les produits correspondants
    results = search_products(ingredients)

    # Afficher les résultats
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, f"Recette trouvée : {selected_recipe['recipe_name']}\n")
    result_text.insert(tk.END, "\nIngrédients nécessaires :\n")
    for ingredient in ingredients:
        result_text.insert(tk.END, f"  - {ingredient.strip()}\n")

    result_text.insert(tk.END, "\nProduits correspondants par magasin :\n")
    for result in results:
        result_text.insert(tk.END, f"\nPour '{result['ingredient']}':\n")
        result_text.insert(tk.END, "  Auchan:\n")
        for product in result['auchan']:
            result_text.insert(tk.END, f"    - {product['nom']} : {product['prix']} € ({product['unite']})\n")
        result_text.insert(tk.END, "  Carrefour:\n")
        for product in result['carrefour']:
            result_text.insert(tk.END, f"    - {product['nom']} : {product['prix']} € ({product['unite']})\n")



### Interface utilisateur

In [89]:
# Interface utilisateur
root = tk.Tk()
root.title("Comparateur de prix de recettes")

# Champ de saisie
recipe_label = ttk.Label(root, text="Entrez le nom de la recette :")
recipe_label.grid(row=0, column=0, padx=10, pady=5, sticky='w')

recipe_entry = ttk.Entry(root, width=50)
recipe_entry.grid(row=1, column=0, padx=10, pady=5)

# Bouton de recherche
search_button = ttk.Button(root, text="Rechercher", command=search_by_recipe_name)
search_button.grid(row=2, column=0, padx=10, pady=10)

# Zone de résultats
result_text = tk.Text(root, height=25, width=80)
result_text.grid(row=3, column=0, padx=10, pady=10)

root.mainloop()
